# Joins

Los joins (funcionalidades para juntar dos o más tablas) son muy esenciales en Spark SQL ya que por lo general se trabaja con diferentes fuentes de datos.

Es necesario comprender los tipos de joins que nos posibilita el Spark SQL y el cómo Spark los ejecuta. El objetivo de este notebook es hacer un tutorial de los diferentes tipos de joins y de las ventajas y limitaciones de cada uno de estos.

## 1. Join en los DataFrames

Los joins en Spark SQL se hacen mediante el método ´join´ del DataFrame de la siguiente manera:

`DataFrame.join(other, on=None, how=None)`

Es necesario definir como argumento principal el dataframe que representaría el lado derecho del join (argumento *other*). Los argumentos opcionales representan el criterio (argumento *on*) y el tipo de join (argumento *how*).

# 2. Elementos de un join

Al momento de hacer un join, es necesario tener en cuenta 2 elementos:
* La condición del join
* El tipo del join



### 2.1 La condición del join

El join también puede ser influenciado por el operador que se utiliza al especificar la condición de un join:
* El operador de igualdad (=), en una operación denominada ***equi join*** (por ejemplo, **cliente.id = transaccion.id**)
* Otros operadores (>, <, >=, <=), en una operación denominada ***non equi join*** (por ejemplo, **cliente.id > transaccion.id**)

Para propósitos de este notebook, nos enfocaremos en operaciones de **equi join**.

### 2.2 El tipo del join

Spark soporta una variedad de tipos de joins. Las más comunes serían:
* Inner Join
* Outer Join
* Semi Join
* Anti Join
* Cross (Cartesian) Join

Ilustraremos con ejemplos cada tipo del join.

In [42]:
# Importar las funciones del Spark SQL
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder \
    .master("local") \
    .appName("Chapter 8 - Joins") \
    .getOrCreate()

In [43]:
# Alterar el número de particiones shuffle
spark.conf.set("spark.sql.shuffle.partitions", 10)

In [44]:
# Cargar los dataframes

customers = spark.read.options(header='True', inferSchema='True').csv("../../data/customers.csv")
transactions = spark.read.options(header='True', inferSchema='True').csv("../../data/transactions.csv")

# Convertir el tipo de la columna "Product" a array
transactions = transactions.withColumn("Product", split(col("Product"), ",").cast("array<string>"))

In [45]:
# Visualizar la tabla de clientes
customers.toPandas()

,Id,CustomerType,FirstName,MiddleName,LastName
0,12,Diamond,Elena,None,Moore
1,13,Bronze,Paul,Williams,Page
2,14,Bronze,Albert,None,Thomas
3,15,Bronze,Sandra,Elizabeth,Faith
4,16,Gold,Robert,None,Alexander


In [46]:
# Visualizar la tabla de transacciones
transactions.toPandas()

,TransactionId,CustomerId,Merchant,Product,TotalAmount,Date
0,1,11,amazon.com.uk,"[shirt, shoes]",150,2021-04-08
1,2,12,marksandspencer.com,"[short, shirt]",50,2021-04-08
2,3,14,amazon.com.uk,"[smartphone, charger]",450,2021-04-09
3,4,12,tesco.com,"[fruits, meat, wholegrains]",75,2021-04-09
4,5,13,apple.com.uk,"[charger, headphone]",120,2021-04-09
5,6,17,e.leclerc,[smartphone],550,2021-04-10
6,7,14,zalando.com,[shoes],45,2021-04-11
7,8,13,zalando.com,"[handbag, jumpsuit]",250,2021-04-13
8,9,15,amazon.com,[books],50,2021-04-13


### 3.1 Inner join

El **inner join** es una expresión que mantiene las filas cuyos valores se encuentran ambas tablas. Si los valores de la columna clave no se muestra en las dos tablas, los respectivos registros no se muestran en el resultado del join.

![img](https://raw.githubusercontent.com/kauvinlucas/Spark-StudyClub/main/Spark%20-%20The%20Definite%20Guide/assets/inner.png)

Por ser una expresión predeterminada, se puede directamente utilizar el join sin necesidad de especificar el tipo.

In [48]:
customers.join(transactions, customers["Id"] == transactions["CustomerId"], how="inner").toPandas()

,Id,CustomerType,FirstName,MiddleName,LastName,TransactionId,CustomerId,Merchant,Product,TotalAmount,Date
0,12,Diamond,Elena,None,Moore,2,12,marksandspencer.com,"[short, shirt]",50,2021-04-08
1,14,Bronze,Albert,None,Thomas,3,14,amazon.com.uk,"[smartphone, charger]",450,2021-04-09
2,12,Diamond,Elena,None,Moore,4,12,tesco.com,"[fruits, meat, wholegrains]",75,2021-04-09
3,13,Bronze,Paul,Williams,Page,5,13,apple.com.uk,"[charger, headphone]",120,2021-04-09
4,14,Bronze,Albert,None,Thomas,7,14,zalando.com,[shoes],45,2021-04-11
5,13,Bronze,Paul,Williams,Page,8,13,zalando.com,"[handbag, jumpsuit]",250,2021-04-13
6,15,Bronze,Sandra,Elizabeth,Faith,9,15,amazon.com,[books],50,2021-04-13


### 3.2 Outer Joins

Retornas las filas que contienen los valores de la clave en ambas, sino retorna valores nulos.

![img](https://github.com/kauvinlucas/Spark-StudyClub/blob/main/Spark%20-%20The%20Definite%20Guide/assets/outer.png?raw=true)

### 3.2.1 Left Outer Join

Retorna todos los registros de la tabla del lado izquierdo del join y los registros del lado derecho que contengan las mismos valores de la clave, sino returna valores nulos del lado derecho.

In [49]:
customers.join(transactions, customers["Id"] == transactions["CustomerId"], how="left_outer").toPandas()

,Id,CustomerType,FirstName,MiddleName,LastName,TransactionId,CustomerId,Merchant,Product,TotalAmount,Date
0,12,Diamond,Elena,None,Moore,4.0,12.0,tesco.com,"[fruits, meat, wholegrains]",75.0,2021-04-09
1,12,Diamond,Elena,None,Moore,2.0,12.0,marksandspencer.com,"[short, shirt]",50.0,2021-04-08
2,13,Bronze,Paul,Williams,Page,8.0,13.0,zalando.com,"[handbag, jumpsuit]",250.0,2021-04-13
3,13,Bronze,Paul,Williams,Page,5.0,13.0,apple.com.uk,"[charger, headphone]",120.0,2021-04-09
4,14,Bronze,Albert,None,Thomas,7.0,14.0,zalando.com,[shoes],45.0,2021-04-11
5,14,Bronze,Albert,None,Thomas,3.0,14.0,amazon.com.uk,"[smartphone, charger]",450.0,2021-04-09
6,15,Bronze,Sandra,Elizabeth,Faith,9.0,15.0,amazon.com,[books],50.0,2021-04-13
7,16,Gold,Robert,None,Alexander,NaN,NaN,None,None,NaN,NaT


### 3.2.1 Right Outer Join

Retorna todos los registros de la tabla del lado derecho del join y los registros del lado izquierdo que contengan las mismos valores de la clave, sino returna valores nulos del lado derecho.

In [51]:
customers.join(transactions, customers["Id"] == transactions["CustomerId"], how="rightouter").toPandas()

,Id,CustomerType,FirstName,MiddleName,LastName,TransactionId,CustomerId,Merchant,Product,TotalAmount,Date
0,NaN,None,None,None,None,1,11,amazon.com.uk,"[shirt, shoes]",150,2021-04-08
1,12.0,Diamond,Elena,None,Moore,2,12,marksandspencer.com,"[short, shirt]",50,2021-04-08
2,14.0,Bronze,Albert,None,Thomas,3,14,amazon.com.uk,"[smartphone, charger]",450,2021-04-09
3,12.0,Diamond,Elena,None,Moore,4,12,tesco.com,"[fruits, meat, wholegrains]",75,2021-04-09
4,13.0,Bronze,Paul,Williams,Page,5,13,apple.com.uk,"[charger, headphone]",120,2021-04-09
5,NaN,None,None,None,None,6,17,e.leclerc,[smartphone],550,2021-04-10
6,14.0,Bronze,Albert,None,Thomas,7,14,zalando.com,[shoes],45,2021-04-11
7,13.0,Bronze,Paul,Williams,Page,8,13,zalando.com,"[handbag, jumpsuit]",250,2021-04-13
8,15.0,Bronze,Sandra,Elizabeth,Faith,9,15,amazon.com,[books],50,2021-04-13


### 3.2.1 Full Outer Join

Retorna todos los registros de la tabla del lado derecho del join y todos los registros del lado izquierdo. Si los valores de la columna clave no coincide con la otra tabla, devuelve valores nulos de la otra tabla.

In [54]:
customers.join(transactions, customers["Id"] == transactions["CustomerId"], how="outer").toPandas()

,Id,CustomerType,FirstName,MiddleName,LastName,TransactionId,CustomerId,Merchant,Product,TotalAmount,Date
0,NaN,None,None,None,None,1.0,11.0,amazon.com.uk,"[shirt, shoes]",150.0,2021-04-08
1,12.0,Diamond,Elena,None,Moore,2.0,12.0,marksandspencer.com,"[short, shirt]",50.0,2021-04-08
2,12.0,Diamond,Elena,None,Moore,4.0,12.0,tesco.com,"[fruits, meat, wholegrains]",75.0,2021-04-09
3,15.0,Bronze,Sandra,Elizabeth,Faith,9.0,15.0,amazon.com,[books],50.0,2021-04-13
4,13.0,Bronze,Paul,Williams,Page,5.0,13.0,apple.com.uk,"[charger, headphone]",120.0,2021-04-09
5,13.0,Bronze,Paul,Williams,Page,8.0,13.0,zalando.com,"[handbag, jumpsuit]",250.0,2021-04-13
6,14.0,Bronze,Albert,None,Thomas,3.0,14.0,amazon.com.uk,"[smartphone, charger]",450.0,2021-04-09
7,14.0,Bronze,Albert,None,Thomas,7.0,14.0,zalando.com,[shoes],45.0,2021-04-11
8,16.0,Gold,Robert,None,Alexander,NaN,NaN,None,None,NaN,NaT
9,NaN,None,None,None,None,6.0,17.0,e.leclerc,[smartphone],550.0,2021-04-10


### 3.3 Left & Right Semi Joins

Los Left Semi Joins funcionan de manera similar al Inner Join, pero muestran solo los registros de la tabla del lado izquierdo.

En otras palabras, evaluan si hay registros con los mismos valores en la columna clave en ambas tablas, y retornan solamente los registros del lado izquierdo del join si hay registros con la misma clave en el lado derecho.

El **Right Semi Join** es una operación que se realiza sobre el lado derecho del join. En este caso, sólo hay que utilizar el método join sobre el otro DataFrame.

![img](https://github.com/kauvinlucas/Spark-StudyClub/blob/main/Spark%20-%20The%20Definite%20Guide/assets/semi.png?raw=true)

In [57]:
# Left Semi Join
customers.join(transactions, customers["Id"] == transactions["CustomerId"], how="left_semi").toPandas()

,Id,CustomerType,FirstName,MiddleName,LastName
0,12,Diamond,Elena,None,Moore
1,13,Bronze,Paul,Williams,Page
2,14,Bronze,Albert,None,Thomas
3,15,Bronze,Sandra,Elizabeth,Faith


In [58]:
# Right Semi Join
transactions.join(customers, transactions["CustomerId"] == customers["Id"], how="left_semi").toPandas()

,TransactionId,CustomerId,Merchant,Product,TotalAmount,Date
0,2,12,marksandspencer.com,"[short, shirt]",50,2021-04-08
1,3,14,amazon.com.uk,"[smartphone, charger]",450,2021-04-09
2,4,12,tesco.com,"[fruits, meat, wholegrains]",75,2021-04-09
3,5,13,apple.com.uk,"[charger, headphone]",120,2021-04-09
4,7,14,zalando.com,[shoes],45,2021-04-11
5,8,13,zalando.com,"[handbag, jumpsuit]",250,2021-04-13
6,9,15,amazon.com,[books],50,2021-04-13


### 3.4 Left & Right Anti Joins

**Los Left Anti Joins son opuestos a los Left Semi Joins**. Evaluan si hay registros con los mismos valores en la columna clave en ambas tablas, pero retornan solamente los registros del lado izquierdo del join si **NO** hay registros correspondientes en el lado derecho.

El Right Anti Join es una operación que se realiza sobre el lado derecho del join. En este caso, sólo hay que utilizar el método join sobre el otro DataFrame.

![img](https://github.com/kauvinlucas/Spark-StudyClub/blob/main/Spark%20-%20The%20Definite%20Guide/assets/anti.png?raw=true)

In [59]:
# Left Anti Join
customers.join(transactions, customers["Id"] == transactions["CustomerId"], how="left_anti").toPandas()

,Id,CustomerType,FirstName,MiddleName,LastName
0,16,Gold,Robert,None,Alexander


In [60]:
# Right Anti Join
transactions.join(customers, transactions["CustomerId"] == customers["Id"], how="left_anti").toPandas()

,TransactionId,CustomerId,Merchant,Product,TotalAmount,Date
0,1,11,amazon.com.uk,"[shirt, shoes]",150,2021-04-08
1,6,17,e.leclerc,[smartphone],550,2021-04-10


### 3.5 Cross Join

Los Cross Joins (o Cartesian Joins) unen cada uno de los registros de una tabla con cada uno de los registros de la otra tabla, haciendo así un producto cartesiano en todos los registros.

Por ejemplo, si deseo juntar una tabla de 1000 registros con otra tabla de 1000 registros, el Cross Join resultaría en 1.000.000 de registros.

Esta "explosión" de registros es la razón por la que Spark internamente opta por hacer un Inner Join en lugar del Cross Join.

In [61]:
# Cross Join
customers.join(transactions, customers["Id"] == transactions["CustomerId"], how="cross").toPandas()

,Id,CustomerType,FirstName,MiddleName,LastName,TransactionId,CustomerId,Merchant,Product,TotalAmount,Date
0,12,Diamond,Elena,None,Moore,2,12,marksandspencer.com,"[short, shirt]",50,2021-04-08
1,14,Bronze,Albert,None,Thomas,3,14,amazon.com.uk,"[smartphone, charger]",450,2021-04-09
2,12,Diamond,Elena,None,Moore,4,12,tesco.com,"[fruits, meat, wholegrains]",75,2021-04-09
3,13,Bronze,Paul,Williams,Page,5,13,apple.com.uk,"[charger, headphone]",120,2021-04-09
4,14,Bronze,Albert,None,Thomas,7,14,zalando.com,[shoes],45,2021-04-11
5,13,Bronze,Paul,Williams,Page,8,13,zalando.com,"[handbag, jumpsuit]",250,2021-04-13
6,15,Bronze,Sandra,Elizabeth,Faith,9,15,amazon.com,[books],50,2021-04-13


Si no especifico la condición del join, Spark no lo haría y retornaría un error:

In [62]:
customers.join(transactions, how="cross").toPandas()

IllegalArgumentException: 'requirement failed: Unsupported using join type Cross'

Para forzar un Cross Join, hay que hacerlo de manera explícita mediante el método **crossJoin**:

In [63]:
customers.crossJoin(transactions).toPandas()

,Id,CustomerType,FirstName,MiddleName,LastName,TransactionId,CustomerId,Merchant,Product,TotalAmount,Date
0,12,Diamond,Elena,None,Moore,1,11,amazon.com.uk,"[shirt, shoes]",150,2021-04-08
1,13,Bronze,Paul,Williams,Page,1,11,amazon.com.uk,"[shirt, shoes]",150,2021-04-08
2,14,Bronze,Albert,None,Thomas,1,11,amazon.com.uk,"[shirt, shoes]",150,2021-04-08
3,15,Bronze,Sandra,Elizabeth,Faith,1,11,amazon.com.uk,"[shirt, shoes]",150,2021-04-08
4,16,Gold,Robert,None,Alexander,1,11,amazon.com.uk,"[shirt, shoes]",150,2021-04-08
5,12,Diamond,Elena,None,Moore,2,12,marksandspencer.com,"[short, shirt]",50,2021-04-08
6,13,Bronze,Paul,Williams,Page,2,12,marksandspencer.com,"[short, shirt]",50,2021-04-08
7,14,Bronze,Albert,None,Thomas,2,12,marksandspencer.com,"[short, shirt]",50,2021-04-08
8,15,Bronze,Sandra,Elizabeth,Faith,2,12,marksandspencer.com,"[short, shirt]",50,2021-04-08
9,16,Gold,Robert,None,Alexander,2,12,marksandspencer.com,"[short, shirt]",50,2021-04-08


In [64]:
transactions.join(customers, transactions["CustomerId"] == customers["Id"], how="left_anti").explain()

== Physical Plan ==
*(2) BroadcastHashJoin [CustomerId#1083], [Id#1062], LeftAnti, BuildRight
:- *(2) Project [TransactionId#1082, CustomerId#1083, Merchant#1084, split(Product#1085, ,) AS Product#1094, TotalAmount#1086, Date#1087]
:  +- *(2) FileScan csv [TransactionId#1082,CustomerId#1083,Merchant#1084,Product#1085,TotalAmount#1086,Date#1087] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/mnt/notebooks/data/transactions.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<TransactionId:int,CustomerId:int,Merchant:string,Product:string,TotalAmount:int,Date:times...
+- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, true] as bigint)))
   +- *(1) Project [Id#1062]
      +- *(1) Filter isnotnull(Id#1062)
         +- *(1) FileScan csv [Id#1062] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/mnt/notebooks/data/customers.csv], PartitionFilters: [], PushedFilters: [IsNotNull(Id)], ReadSchema: struct<Id:int>
